# Metrics for hierarchical graph clustering

This notebook presents the experiments related to the paper:

T. Bonald, B. Charpentier, [Learning Graph Representations by Dendrograms](http://arxiv.org/abs/1807.05087), 2018

These experiments illustrate the properties of the following metric for assessing the quality of a binary tree ${\cal T}$ for representing the hierarchical structure of a graph:
$$
 \sum_{A,B: (A,B) \in {\cal I}}p(A,B) \log \frac{p(A,B)}{\pi(A) \pi(B)},
$$
where:
* ${\cal I}$ is the set of internal nodes of the tree ${\cal T}$ 
* $A,B$ are the sets of nodes induced by each element of ${\cal I}$
* $p(A,B)$ is the sampling probability of the node sets $A,B$
* $\pi(A)$ is the sampling probability of the node set $A$

This metric is the Kullback-Leibler divergence between the  probability distribution on node sets  induced by the tree ${\cal T}$ and that induced by independent node sampling from the   distribution $\pi$. 

Another popular metric for assessing the quality of a binary tree ${\cal T}$ is Dasgupta's cost function:
$$
\sum_{A,B: (A,B) \in {\cal I}}p(A,B) (\pi(A)  + \pi(B)).
$$

## Import

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

## Algorithms

In [ ]:
def init_aggregate_graph(graph):
    aggregate_graph = graph.copy()
    # index nodes from 0 to n - 1
    number_nodes = aggregate_graph.number_of_nodes()
    if set(aggregate_graph.nodes()) != set(range(number_nodes)):
        aggregate_graph = nx.convert_node_labels_to_integers(aggregate_graph)
    # node weights
    node_weights = {u: 0. for u in range(number_nodes)}
    total_weight = 0. 
    edges = list(aggregate_graph.edges())
    for (u,v) in edges:
        if u == v:
            # remove self-loop
            aggregate_graph.remove_edge(u,u)
        else:
            if 'weight' not in aggregate_graph[u][v]:
                aggregate_graph[u][v]['weight'] = 1.
            weight = aggregate_graph[u][v]['weight']
            node_weights[u] += weight
            node_weights[v] += weight
            total_weight += 2 * weight
    nx.set_node_attributes(aggregate_graph, node_weights, 'weight')
    # node sizes   
    nx.set_node_attributes(aggregate_graph, 1, 'size')
    return aggregate_graph,total_weight

In [ ]:
def merge_nodes(graph, u, v, new_node):
    neighbors_u = list(graph.neighbors(u))
    neighbors_v = list(graph.neighbors(v))
    graph.add_node(new_node)
    for node in neighbors_u:
        graph.add_edge(new_node,node,weight = graph[u][node]['weight'])
    for node in neighbors_v:
        if graph.has_edge(new_node,node):
            graph[new_node][node]['weight'] += graph[v][node]['weight']
        else:
            graph.add_edge(new_node,node,weight = graph[v][node]['weight'])
    graph.node[new_node]['weight'] = graph.node[u]['weight'] + graph.node[v]['weight']
    graph.node[new_node]['size'] = graph.node[u]['size'] + graph.node[v]['size']
    graph.remove_node(u)
    graph.remove_node(v)
    return graph

In [ ]:
def paris_hierarchy(graph):
    # dendrogram as list of merges
    dendrogram = []
    
    if nx.is_connected(graph):
        aggregate_graph, total_weight = init_aggregate_graph(graph)
        number_nodes = aggregate_graph.number_of_nodes()
        new_node = number_nodes
        while number_nodes > 0:
            # nearest-neighbor chain
            chain = [list(aggregate_graph.nodes())[0]]
            while chain != []:
                current_node = chain.pop()
                # find nearest neighbor 
                distance_min = float("inf")
                nearest_neighbor = -1
                for node in aggregate_graph.neighbors(current_node):
                    if node != current_node:
                        distance = (aggregate_graph.node[current_node]['weight'] * aggregate_graph.node[node]['weight'] 
                            / aggregate_graph[current_node][node]['weight'] / total_weight)
                        if distance < distance_min:
                            nearest_neighbor = node
                            distance_min = distance
                        elif distance == distance_min:
                            nearest_neighbor = min(nearest_neighbor,node)
                distance = distance_min
                if chain != []:
                    next_node = chain.pop()
                    if next_node == nearest_neighbor:
                        # merge nodes
                        size = aggregate_graph.node[current_node]['size'] + aggregate_graph.node[next_node]['size']
                        dendrogram.append([current_node,next_node,distance,size])
                        aggregate_graph = merge_nodes(aggregate_graph,current_node,next_node,new_node)
                        number_nodes -= 1
                        new_node += 1
                    else:
                        chain.append(next_node)
                        chain.append(current_node)
                        chain.append(nearest_neighbor)
                elif nearest_neighbor >= 0:
                    chain.append(current_node)
                    chain.append(nearest_neighbor)
                else:
                    number_nodes -= 1
    else:
        print("Error: The graph is not connected")
    return np.array(dendrogram, float)

In [ ]:
def newman_hierarchy(graph):
    # dendrogram as list of merges
    dendrogram = []
    
    if nx.is_connected(graph):
        aggregate_graph, total_weight = init_aggregate_graph(graph)
        # modularity increase
        for (u,v) in aggregate_graph.edges():
            aggregate_graph[u][v]['delta'] = 2 * (aggregate_graph[u][v]['weight']  
                                                  - aggregate_graph.node[u]['weight'] 
                                                  * aggregate_graph.node[v]['weight'] 
                                                  / total_weight) / total_weight
        number_nodes = aggregate_graph.number_of_nodes()
        new_node = number_nodes
        while number_nodes > 1:
            # find the best node pair for modularity increase
            delta = nx.get_edge_attributes(aggregate_graph,'delta')
            u,v = max(delta, key = delta.get)
            if u == v:
                print(u,aggregate_graph[u][u]['delta'])
            # merge nodes
            size = aggregate_graph.node[u]['size'] + aggregate_graph.node[v]['size']
            dendrogram.append([u,v,size,size])
            aggregate_graph = merge_nodes(aggregate_graph,u,v,new_node)
            for u in aggregate_graph.neighbors(new_node):
                aggregate_graph[u][new_node]['delta'] = 2 * (aggregate_graph[u][new_node]['weight']  
                                                  - aggregate_graph.node[u]['weight'] 
                                                  * aggregate_graph.node[new_node]['weight'] 
                                                  / total_weight) / total_weight
            number_nodes -= 1
            new_node += 1
    else:
        print("Error: The graph is not connected")
    return np.array(dendrogram, float)    

In [ ]:
def random_hierarchy(graph):
    # dendrogram as list of merges
    dendrogram = []
    
    if nx.is_connected(graph):
        aggregate_graph, total_weight = init_aggregate_graph(graph)
        number_nodes = aggregate_graph.number_of_nodes()
        new_node = number_nodes
        while number_nodes > 1:
            # random edge 
            edges = list(aggregate_graph.edges())
            u,v = edges[np.random.randint(len(edges))]
            # merge nodes
            size = aggregate_graph.node[u]['size'] + aggregate_graph.node[v]['size']
            dendrogram.append([u,v,size,size])
            aggregate_graph = merge_nodes(aggregate_graph,u,v,new_node)
            number_nodes -= 1
            new_node += 1
    else:
        print("Error: The graph is not connected")
    return np.array(dendrogram, float)    

In [ ]:
def hierarchical_clustering(graph, algorithm):
    if algorithm == "paris":
        return paris_hierarchy(graph)
    elif algorithm == "newman":
        return newman_hierarchy(graph)
    elif algorithm == "random":
        return random_hierarchy(graph)
    else:
        print("Unknown algorithm")

## Metrics

In [ ]:
def relative_entropy(graph, dendrogram, weighted = True): 
    aggregate_graph, total_weight = init_aggregate_graph(graph) 
    number_nodes = aggregate_graph.number_of_nodes()
    if weighted:
        pi = {u: aggregate_graph.node[u]['weight'] / total_weight for u in aggregate_graph.nodes()}
    else:
        pi = {u: 1. / number_nodes for u in aggregate_graph.nodes()}
    quality = 0.
    for t in range(number_nodes - 1):
        u = int(dendrogram[t][0])
        v = int(dendrogram[t][1])
        if aggregate_graph.has_edge(u,v):
            p = 2 * aggregate_graph[u][v]['weight'] / total_weight 
            quality += p * np.log(p / pi[u] / pi[v])
        aggregate_graph = merge_nodes(aggregate_graph, u, v, number_nodes + t)
        pi[number_nodes + t] = pi.pop(u) + pi.pop(v)
    return quality  

In [ ]:
def dasgupta_cost(graph, dendrogram, weighted = True):    
    aggregate_graph, total_weight = init_aggregate_graph(graph) 
    number_nodes = aggregate_graph.number_of_nodes()
    if weighted:
        pi = {u: aggregate_graph.node[u]['weight'] / total_weight for u in aggregate_graph.nodes()}
    else:
        pi = {u: 1. / number_nodes for u in aggregate_graph.nodes()}
    cost = 0.
    for t in range(number_nodes - 1):
        u = int(dendrogram[t][0])
        v = int(dendrogram[t][1])
        if aggregate_graph.has_edge(u,v):
            p = 2 * aggregate_graph[u][v]['weight'] / total_weight 
            cost += p * (pi[u] + pi[v])
        aggregate_graph = merge_nodes(aggregate_graph, u, v, number_nodes + t)
        pi[number_nodes + t] = pi.pop(u) + pi.pop(v)
    return cost    

## Experiments on real data

In [ ]:
import urllib.request

url = "http://perso.telecom-paristech.fr/~bonald/graphs/"

# Openflights
dataset = "openflights.graphml.gz"
# Wikipedia for schools
#dataset = "wikipedia_schools_undirected.graphml.gz"

download = urllib.request.urlretrieve(url + dataset, dataset)

In [ ]:
graph = nx.read_graphml(dataset, node_type=int)

In [ ]:
print(nx.info(graph))

In [ ]:
# Number of samples for the random algorithm
number_samples = 100

In [ ]:
dendrogram_paris = hierarchical_clustering(graph, "paris")

In [ ]:
dendrogram_newman = hierarchical_clustering(graph, "newman")

In [ ]:
dendrogram_random = [hierarchical_clustering(graph, "random") for s in range(number_samples)]

In [ ]:
print('Relative entropy (weighted, uniform)')
print('Paris hierarchy: ', relative_entropy(graph, dendrogram_paris), relative_entropy(graph, dendrogram_paris, False))
print('Newman hierarchy: ', relative_entropy(graph, dendrogram_newman), relative_entropy(graph, dendrogram_newman, False))
print('Random hierarchy: ', np.mean([relative_entropy(graph, d) for d in dendrogram_random]),np.mean([relative_entropy(graph, d, False) for d in dendrogram_random]))

In [ ]:
print('Dasgupta cost (weighted, uniform)')
print('Paris hierarchy: ', dasgupta_cost(graph, dendrogram_paris), dasgupta_cost(graph, dendrogram_paris, False))
print('Newman hierarchy: ', dasgupta_cost(graph, dendrogram_newman), dasgupta_cost(graph, dendrogram_newman, False))
print('Random hierarchy: ', np.mean([dasgupta_cost(graph, d) for d in dendrogram_random]), np.mean([dasgupta_cost(graph, d, False) for d in dendrogram_random])) 

## Experiments on synthetic data

In [ ]:
def random_dendrogram(number_nodes = 100):
    nodes = list(range(number_nodes))
    dendrogram = []
    t = 0
    size = {u: 1 for u in nodes}
    while (len(nodes)) > 1:
        u = nodes.pop(np.random.randint(len(nodes)))
        v = nodes.pop(np.random.randint(len(nodes)))
        new_node = number_nodes + t
        t += 1
        size[new_node] = size.pop(u) + size.pop(v)
        dendrogram.append([u,v,size[new_node],size[new_node]])
        nodes.append(new_node)
    return np.array(dendrogram, float)

In [ ]:
def get_similarity(dendrogram):
    n = np.shape(dendrogram)[0] + 1
    sim = np.zeros((n,n),float)
    cluster = {u:[u] for u in range(n)}
    for t in range(n - 1):
        u = int(dendrogram[t][0])
        v = int(dendrogram[t][1])
        for i in cluster[u]:
            for j in cluster[v]:
                sim[i][j] = 1 / dendrogram[t][2]
        cluster[n + t] = cluster.pop(u) + cluster.pop(v)
    return sim

In [ ]:
def generate_graph(dendrogram, average_degree = 10):
    n = np.shape(dendrogram)[0] + 1
    similarity = get_similarity(dendrogram)
    is_connected = False
    while not is_connected:
        adjacency = np.random.rand(n,n) < similarity / np.sum(similarity) * n * average_degree / 2
        adjacency = np.array(adjacency + adjacency.T,int)
        graph = nx.from_numpy_matrix(adjacency)
        is_connected = nx.is_connected(graph)
    return graph

In [ ]:
def add_noise(graph, prob = 0.1):
    is_connected = False
    while not is_connected:
        new_graph = graph.copy()
        edges = list(graph.edges())
        indices = np.random.choice(list(range(len(edges))),replace = False, size = int(np.floor(prob * len(edges))))
        for i in indices:
            u,v = edges[i]
            new_graph.remove_edge(u,v)
            new_edge = np.random.choice(list(new_graph.nodes()), replace = False, size = 2)
            new_graph.add_edge(new_edge[0],new_edge[1],weight = 1.)
        is_connected = nx.is_connected(new_graph)
    return new_graph

In [ ]:
def classification_scores(number_nodes, average_degree, prob_range, number_samples, algorithm, weighted = True):
    results = []
    for prob in prob_range:
        cost = 0.
        quality = 0.
        for s in range(number_samples):
            dendrogram = random_dendrogram(number_nodes)
            graph = generate_graph(dendrogram, average_degree)
            graph1 = add_noise(graph,prob)
            graph2 = add_noise(graph,prob)
            dendrogram1 = hierarchical_clustering(graph1, algorithm)
            dendrogram2 = hierarchical_clustering(graph2, algorithm)
            cost += (dasgupta_cost(graph1, dendrogram1, weighted) < dasgupta_cost(graph1, dendrogram2, weighted))
            cost += (dasgupta_cost(graph2, dendrogram2, weighted) < dasgupta_cost(graph2, dendrogram1, weighted))
            quality += (relative_entropy(graph1, dendrogram1, weighted) > relative_entropy(graph1, dendrogram2, weighted))
            quality += (relative_entropy(graph2, dendrogram2, weighted) > relative_entropy(graph2, dendrogram1, weighted))
        results.append((cost / 2 / number_samples, quality / 2 / number_samples))
    return np.array(results)

In [ ]:
number_nodes = 100
average_degree = 10
prob_range = np.arange(0.01,0.2,0.03)
number_samples = 1000
results_paris = classification_scores(number_nodes, average_degree, prob_range, number_samples, "paris", True)
results_newman = classification_scores(number_nodes, average_degree, prob_range, number_samples, "newman", True)

In [ ]:
plt.figure()
plt.plot(100 * prob_range,100 * results_paris[:,1],label = 'Entropy', color = "b")
plt.plot(100 * prob_range,100 * results_paris[:,0],'--',label = 'Dasgupta',color = "b")
plt.plot(100 * prob_range,100 * results_newman[:,1], color = "r")
plt.plot(100 * prob_range,100 * results_newman[:,0],'--',color = "r")
plt.xticks(np.arange(0, 21, step=5))
plt.xlabel("Graph distance (%)")
plt.ylabel("Classification score (%)")
plt.legend()
plt.show()